In [ ]:
model_name = 'distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european'

In [4]:
import wandb

wandb.login()

True

In [5]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'data/train_subset.csv', 'eval': 'data/eval_subset.csv'})

In [6]:
from utils import transform_labels
dataset = dataset.map(transform_labels)

In [7]:
# let's train a Distilbert model

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example['Sentence'], padding='max_length')    

In [8]:
dataset = dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# the default batch size for training arguments
batch_size = 8

# set number of epochs
number_of_epochs = 7
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch 

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.2 * steps)

In [ ]:
from transformers import TrainingArguments
from utils import get_output_dir
training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  eval_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir=get_output_dir(model_name),
                                  report_to="wandb"
                                )

e:\repo\DistilBERTFinancialSentiment\.venv\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['eval'].shuffle(seed=10)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [14]:
# Launch the learning process: training
# Load model from checkpoint
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
500,0.994900,0.902183
1000,0.884200,0.872294
1500,0.693900,0.934768
2000,0.269900,1.257892
2500,0.188400,1.528810


TrainOutput(global_step=2800, training_loss=0.5161811992100307, metrics={'train_runtime': 335.3428, 'train_samples_per_second': 66.797, 'train_steps_per_second': 8.35, 'total_flos': 2967322646937600.0, 'train_loss': 0.5161811992100307, 'epoch': 7.0})

In [15]:
from utils import compute_metrics

trainer_eval = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [16]:
trainer_eval.evaluate()

{'eval_loss': 0.8722943067550659,
 'eval_model_preparation_time': 0.0025,
 'eval_accuracy': {'accuracy': 0.595},
 'eval_f1score': {'f1': 0.5920038067436475},
 'eval_runtime': 5.5486,
 'eval_samples_per_second': 144.181,
 'eval_steps_per_second': 18.023}

In [17]:
model.save_pretrained(model_name)
model.push_to_hub(f'nojedag/{model_name}')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/distilbert-base-uncased_financial_fr_gr/commit/9547a1bd8f3733a580c763413e7322ca85f9998e', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='9547a1bd8f3733a580c763413e7322ca85f9998e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/distilbert-base-uncased_financial_fr_gr', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/distilbert-base-uncased_financial_fr_gr'), pr_revision=None, pr_num=None)

In [18]:
trainer.push_to_hub()
trainer_eval.push_to_hub()

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/distilbert-base-uncased_financial_fr_gr/commit/e1426c483cdc718041337b1defd6d3f68b1c8302', commit_message='End of training', commit_description='', oid='e1426c483cdc718041337b1defd6d3f68b1c8302', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/distilbert-base-uncased_financial_fr_gr', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/distilbert-base-uncased_financial_fr_gr'), pr_revision=None, pr_num=None)

In [ ]:
wandb.finish()